In [13]:
import sys,os
import matplotlib
sys.path.append(os.path.join(os.environ['ATS_SRC_DIR'],'tools', 'utils'))
import parse_ats
import column_data
import colors
import numpy as np
from matplotlib import pyplot as plt

%matplotlib

Using matplotlib backend: MacOSX


In [16]:
def getFigs():
    fig,axs = plt.subplots(nrows=3, ncols=2)
    axs = axs.reshape((-1,))
    return fig,axs

# collect data, determine control for plots
ranges = [ (1.,2.), (10.,11.),]# (100.,101.)]#, (1000.,1001.) ]
clist = [ 'blue', 'green', 'goldenrod', 'red']

# create figures
fig, axs = getFigs()

# get colormap
cm = colors.cm_mapper(0., 1.)

# plot the temperatures and co2 decomp on the last year
keys, times, fid = parse_ats.readATS(time_range=ranges[-1], directory="vegetation-spinup.regression.gold")
fid.close()
to_get = ['temperature.cell.0', 'co2_decomposition.cell.0']
dat = column_data.column_data(to_get, keys, directory="vegetation-spinup.regression.gold")
z = dat[0,0,:]

# plot the vegetation at each year
for c, time_range in zip(clist,ranges):
    # surface plots
    skeys, stimes, sdat = parse_ats.readATS(base='visdump_surface_data.h5', 
                                            directory="vegetation-spinup.regression.gold", 
                                            time_range=time_range)

    # convert times to days
    times = [(time-time_range[0])*365.25 for time in stimes]

    bio_moss = parse_ats.getSurfaceData(skeys, sdat, 'total_biomass.cell.1')
    bio_sedge = parse_ats.getSurfaceData(skeys, sdat, 'total_biomass.cell.0')
    bioleaf_moss = parse_ats.getSurfaceData(skeys, sdat, 'leaf_biomass.cell.moss')
    bioleaf_sedge = parse_ats.getSurfaceData(skeys, sdat, 'leaf_biomass.cell.sedge')
    lai_moss = parse_ats.getSurfaceData(skeys, sdat, 'leaf_area_index.cell.1')
    lai_sedge = parse_ats.getSurfaceData(skeys, sdat, 'leaf_area_index.cell.0')
    et_moss = parse_ats.getSurfaceData(skeys, sdat, 'veg_total_transpiration.cell.1')
    et_sedge = parse_ats.getSurfaceData(skeys, sdat, 'veg_total_transpiration.cell.0')
    
    axs[0].plot(times, bio_moss, color=c, label="total, year %d"%time_range[1])
    axs[1].plot(times, bio_sedge, color=c, label="total, year %d"%time_range[1])
    axs[0].plot(times, bioleaf_moss, '--', color=c, label="leaf, year %d"%time_range[1])
    axs[1].plot(times, bioleaf_sedge, '--', color=c, label="leaf, year %d"%time_range[1])
    axs[2].plot(times, lai_moss, color=c, label="year %d"%time_range[1])
    axs[3].plot(times, lai_sedge, color=c, label="year %d"%time_range[1])
    axs[4].plot(times, et_moss, color=c, label= "year %d"%time_range[1])
    axs[5].plot(times, et_sedge, color=c, label="year %d"%time_range[1])
    sdat.close()

axs[0].set_ylabel("biomass of moss")
axs[0].set_xlabel("time [d]")
axs[1].set_ylabel("biomass of sedge")
axs[1].set_xlabel("time [d]")
axs[2].set_ylabel("LAI of moss")
axs[2].set_xlabel("time [d]")
axs[3].set_ylabel("LAI of sedge")
axs[3].set_xlabel("time [d]")
axs[4].set_ylabel("ET of moss(kg H2O/cell/day)")
axs[4].set_xlabel("time [d]")
axs[5].set_ylabel("ET of sedge(kg H2O/cell/day)")
axs[5].set_xlabel("time [d]")
axs[1].legend()
axs[0].legend()
axs[2].legend()

plt.tight_layout()
#plt.savefig('soln_temp.pdf')
plt.show()
